In [1]:
import pandas as pd
import ccxt

## Data obtaining from ccxt library 
--------------------------------------------
motivation, get high frequency data from exchange
exchange(s) should be decided \
Another thing is that cryptocurrency against which currency should be decided. \
anlamadığım bir durum var doları cryptocurrency -lere karşı bulamıyorum

In [ ]:
#https://backtest-rookies.com/2018/03/08/download-cryptocurrency-data-with-ccxt/
#https://www.linkedin.com/pulse/download-historical-data-all-cryptocoins-ccxt-gursel-karacor/?articleId=6636391574811090944   
## final: https://github.com/codeninja/CCXT-Historical-Data/blob/master/Binance%20Historical%20Data%20.ipynb

In [2]:
import os
from pathlib import Path

import sys
import csv

import ccxt
# -----------------------------------------------------------------------------

def retry_fetch_ohlcv(exchange, max_retries, symbol, timeframe, since, limit):
    num_retries = 0
    try:
        num_retries += 1
        ohlcv = exchange.fetch_ohlcv(symbol, timeframe, since, limit)
        # print('Fetched', len(ohlcv), symbol, 'candles from', exchange.iso8601 (ohlcv[0][0]), 'to', exchange.iso8601 (ohlcv[-1][0]))
        return ohlcv
    except Exception:
        if num_retries > max_retries:
            raise  # Exception('Failed to fetch', timeframe, symbol, 'OHLCV in', max_retries, 'attempts')


def scrape_ohlcv(exchange, max_retries, symbol, timeframe, since, limit):
    earliest_timestamp = exchange.milliseconds()
    timeframe_duration_in_seconds = exchange.parse_timeframe(timeframe)
    timeframe_duration_in_ms = timeframe_duration_in_seconds * 1000
    timedelta = limit * timeframe_duration_in_ms
    all_ohlcv = []
    while True:
        fetch_since = earliest_timestamp - timedelta
        ohlcv = retry_fetch_ohlcv(exchange, max_retries, symbol, timeframe, fetch_since, limit)
        # if we have reached the beginning of history
        if len(ohlcv) == 0:
            break
        if ohlcv[0][0] >= earliest_timestamp:
            break
        earliest_timestamp = ohlcv[0][0]
        all_ohlcv = ohlcv + all_ohlcv
        print(len(all_ohlcv), symbol, 'candles in total from', exchange.iso8601(all_ohlcv[0][0]), 'to', exchange.iso8601(all_ohlcv[-1][0]))
        # if we have reached the checkpoint
        if fetch_since < since:
            break
    return all_ohlcv


def write_to_csv(filename, exchange, data):
    p = Path("../data/0_raw/", str(exchange))
    p.mkdir(parents=True, exist_ok=True)
    full_path = p / str(filename)
    with Path(full_path).open('w+', newline='') as output_file:
        csv_writer = csv.writer(output_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        csv_writer.writerows(data)


def scrape_candles_to_csv(filename, exchange_id, max_retries, symbol, timeframe, since, limit):
    # instantiate the exchange by id
    exchange = getattr(ccxt, exchange_id)({
        'enableRateLimit': True,  # required by the Manual
    })
    # convert since from string to milliseconds integer if needed
    if isinstance(since, str):
        since = exchange.parse8601(since)
    # preload all markets from the exchange
    exchange.load_markets()
    # fetch all candles+
    ohlcv = scrape_ohlcv(exchange, max_retries, symbol, timeframe, since, limit)
    # save them to csv file
    write_to_csv(filename, exchange, ohlcv)
    print('Saved', len(ohlcv), 'candles from', exchange.iso8601(ohlcv[0][0]), 'to', exchange.iso8601(ohlcv[-1][0]), 'to', filename)

In [ ]:
#BUSD ile çok az veri çekilebildiği için usdt yi de çektim.

In [3]:
scrape_candles_to_csv('./btc_usdt_1h.csv', 'binance', 3, 'BTC/USDT', '1h', '2017-01-0100:00:00Z', 1000)

996 BTC/USDT candles in total from 2021-07-05T13:00:00.000Z to 2021-08-16T04:00:00.000Z
1996 BTC/USDT candles in total from 2021-05-24T21:00:00.000Z to 2021-08-16T04:00:00.000Z
2991 BTC/USDT candles in total from 2021-04-13T05:00:00.000Z to 2021-08-16T04:00:00.000Z
3990 BTC/USDT candles in total from 2021-03-02T13:00:00.000Z to 2021-08-16T04:00:00.000Z
4989 BTC/USDT candles in total from 2021-01-19T21:00:00.000Z to 2021-08-16T04:00:00.000Z
5985 BTC/USDT candles in total from 2020-12-09T05:00:00.000Z to 2021-08-16T04:00:00.000Z
6984 BTC/USDT candles in total from 2020-10-28T13:00:00.000Z to 2021-08-16T04:00:00.000Z
7984 BTC/USDT candles in total from 2020-09-16T21:00:00.000Z to 2021-08-16T04:00:00.000Z
8984 BTC/USDT candles in total from 2020-08-06T05:00:00.000Z to 2021-08-16T04:00:00.000Z
9981 BTC/USDT candles in total from 2020-06-25T13:00:00.000Z to 2021-08-16T04:00:00.000Z
10981 BTC/USDT candles in total from 2020-05-14T21:00:00.000Z to 2021-08-16T04:00:00.000Z
11979 BTC/USDT candle

In [4]:
scrape_candles_to_csv('./eth_usdt_1h.csv', 'binance', 3, 'ETH/USDT', '1h', '2017-01-0100:00:00Z', 1000)

996 ETH/USDT candles in total from 2021-07-05T13:00:00.000Z to 2021-08-16T04:00:00.000Z
1996 ETH/USDT candles in total from 2021-05-24T21:00:00.000Z to 2021-08-16T04:00:00.000Z
2991 ETH/USDT candles in total from 2021-04-13T05:00:00.000Z to 2021-08-16T04:00:00.000Z
3990 ETH/USDT candles in total from 2021-03-02T13:00:00.000Z to 2021-08-16T04:00:00.000Z
4989 ETH/USDT candles in total from 2021-01-19T21:00:00.000Z to 2021-08-16T04:00:00.000Z
5985 ETH/USDT candles in total from 2020-12-09T05:00:00.000Z to 2021-08-16T04:00:00.000Z
6984 ETH/USDT candles in total from 2020-10-28T13:00:00.000Z to 2021-08-16T04:00:00.000Z
7984 ETH/USDT candles in total from 2020-09-16T21:00:00.000Z to 2021-08-16T04:00:00.000Z
8984 ETH/USDT candles in total from 2020-08-06T05:00:00.000Z to 2021-08-16T04:00:00.000Z
9981 ETH/USDT candles in total from 2020-06-25T13:00:00.000Z to 2021-08-16T04:00:00.000Z
10981 ETH/USDT candles in total from 2020-05-14T21:00:00.000Z to 2021-08-16T04:00:00.000Z
11979 ETH/USDT candle

In [5]:
scrape_candles_to_csv('./ltc_usdt_1h.csv', 'binance', 3, 'LTC/USDT', '1h', '2017-01-0100:00:00Z', 1000)

996 LTC/USDT candles in total from 2021-07-05T13:00:00.000Z to 2021-08-16T04:00:00.000Z
1996 LTC/USDT candles in total from 2021-05-24T21:00:00.000Z to 2021-08-16T04:00:00.000Z
2991 LTC/USDT candles in total from 2021-04-13T05:00:00.000Z to 2021-08-16T04:00:00.000Z
3990 LTC/USDT candles in total from 2021-03-02T13:00:00.000Z to 2021-08-16T04:00:00.000Z
4989 LTC/USDT candles in total from 2021-01-19T21:00:00.000Z to 2021-08-16T04:00:00.000Z
5985 LTC/USDT candles in total from 2020-12-09T05:00:00.000Z to 2021-08-16T04:00:00.000Z
6984 LTC/USDT candles in total from 2020-10-28T13:00:00.000Z to 2021-08-16T04:00:00.000Z
7984 LTC/USDT candles in total from 2020-09-16T21:00:00.000Z to 2021-08-16T04:00:00.000Z
8984 LTC/USDT candles in total from 2020-08-06T05:00:00.000Z to 2021-08-16T04:00:00.000Z
9981 LTC/USDT candles in total from 2020-06-25T13:00:00.000Z to 2021-08-16T04:00:00.000Z
10981 LTC/USDT candles in total from 2020-05-14T21:00:00.000Z to 2021-08-16T04:00:00.000Z
11979 LTC/USDT candle

In [6]:
for filename in ["btc_usdt_1h.csv", "eth_usdt_1h.csv", "ltc_usdt_1h.csv"]:
    df = pd.read_csv('../data/0_raw/Binance/' + filename, header=None)
    df.columns = ['Timestamp', 'Open', 'High', 'Low', 'Close', 'Volume']
    df.set_index('Timestamp', inplace=True)
    df.index = df.index /1000
    df["Date"] = pd.to_datetime(df.index,unit='s')
    df.to_csv("../data/0_raw/" + filename)

In [ ]:
#https://coinmarketcap.com/all/views/all/

In [7]:
scrape_candles_to_csv('./ada_usdt_1h.csv', 'binance', 3, 'ADA/USDT', '1h', '2017-01-0100:00:00Z', 1000)

996 ADA/USDT candles in total from 2021-07-05T13:00:00.000Z to 2021-08-16T04:00:00.000Z
1996 ADA/USDT candles in total from 2021-05-24T21:00:00.000Z to 2021-08-16T04:00:00.000Z
2991 ADA/USDT candles in total from 2021-04-13T05:00:00.000Z to 2021-08-16T04:00:00.000Z
3990 ADA/USDT candles in total from 2021-03-02T13:00:00.000Z to 2021-08-16T04:00:00.000Z
4989 ADA/USDT candles in total from 2021-01-19T21:00:00.000Z to 2021-08-16T04:00:00.000Z
5985 ADA/USDT candles in total from 2020-12-09T05:00:00.000Z to 2021-08-16T04:00:00.000Z
6984 ADA/USDT candles in total from 2020-10-28T13:00:00.000Z to 2021-08-16T04:00:00.000Z
7984 ADA/USDT candles in total from 2020-09-16T21:00:00.000Z to 2021-08-16T04:00:00.000Z
8984 ADA/USDT candles in total from 2020-08-06T05:00:00.000Z to 2021-08-16T04:00:00.000Z
9981 ADA/USDT candles in total from 2020-06-25T13:00:00.000Z to 2021-08-16T04:00:00.000Z
10981 ADA/USDT candles in total from 2020-05-14T21:00:00.000Z to 2021-08-16T04:00:00.000Z
11979 ADA/USDT candle

In [8]:
scrape_candles_to_csv('./xrp_usdt_1h.csv', 'binance', 3, 'XRP/USDT', '1h', '2017-01-0100:00:00Z', 1000)

996 XRP/USDT candles in total from 2021-07-05T13:00:00.000Z to 2021-08-16T04:00:00.000Z
1996 XRP/USDT candles in total from 2021-05-24T21:00:00.000Z to 2021-08-16T04:00:00.000Z
2991 XRP/USDT candles in total from 2021-04-13T05:00:00.000Z to 2021-08-16T04:00:00.000Z
3990 XRP/USDT candles in total from 2021-03-02T13:00:00.000Z to 2021-08-16T04:00:00.000Z
4989 XRP/USDT candles in total from 2021-01-19T21:00:00.000Z to 2021-08-16T04:00:00.000Z
5985 XRP/USDT candles in total from 2020-12-09T05:00:00.000Z to 2021-08-16T04:00:00.000Z
6984 XRP/USDT candles in total from 2020-10-28T13:00:00.000Z to 2021-08-16T04:00:00.000Z
7984 XRP/USDT candles in total from 2020-09-16T21:00:00.000Z to 2021-08-16T04:00:00.000Z
8984 XRP/USDT candles in total from 2020-08-06T05:00:00.000Z to 2021-08-16T04:00:00.000Z
9981 XRP/USDT candles in total from 2020-06-25T13:00:00.000Z to 2021-08-16T04:00:00.000Z
10981 XRP/USDT candles in total from 2020-05-14T21:00:00.000Z to 2021-08-16T04:00:00.000Z
11979 XRP/USDT candle

In [9]:
scrape_candles_to_csv('./doge_usdt_1h.csv', 'binance', 3, 'DOGE/USDT', '1h', '2017-01-0100:00:00Z', 1000)

996 DOGE/USDT candles in total from 2021-07-05T13:00:00.000Z to 2021-08-16T04:00:00.000Z
1996 DOGE/USDT candles in total from 2021-05-24T21:00:00.000Z to 2021-08-16T04:00:00.000Z
2991 DOGE/USDT candles in total from 2021-04-13T05:00:00.000Z to 2021-08-16T04:00:00.000Z
3990 DOGE/USDT candles in total from 2021-03-02T13:00:00.000Z to 2021-08-16T04:00:00.000Z
4989 DOGE/USDT candles in total from 2021-01-19T21:00:00.000Z to 2021-08-16T04:00:00.000Z
5985 DOGE/USDT candles in total from 2020-12-09T05:00:00.000Z to 2021-08-16T04:00:00.000Z
6984 DOGE/USDT candles in total from 2020-10-28T13:00:00.000Z to 2021-08-16T04:00:00.000Z
7984 DOGE/USDT candles in total from 2020-09-16T21:00:00.000Z to 2021-08-16T04:00:00.000Z
8984 DOGE/USDT candles in total from 2020-08-06T05:00:00.000Z to 2021-08-16T04:00:00.000Z
9981 DOGE/USDT candles in total from 2020-06-25T13:00:00.000Z to 2021-08-16T04:00:00.000Z
10981 DOGE/USDT candles in total from 2020-05-14T21:00:00.000Z to 2021-08-16T04:00:00.000Z
11979 DOGE

In [10]:
scrape_candles_to_csv('./dot_usdt_1h.csv', 'binance', 3, 'DOT/USDT', '1h', '2017-01-0100:00:00Z', 1000) 
#bu azmış çıkarabiliriz

996 DOT/USDT candles in total from 2021-07-05T13:00:00.000Z to 2021-08-16T04:00:00.000Z
1996 DOT/USDT candles in total from 2021-05-24T21:00:00.000Z to 2021-08-16T04:00:00.000Z
2991 DOT/USDT candles in total from 2021-04-13T05:00:00.000Z to 2021-08-16T04:00:00.000Z
3990 DOT/USDT candles in total from 2021-03-02T13:00:00.000Z to 2021-08-16T04:00:00.000Z
4989 DOT/USDT candles in total from 2021-01-19T21:00:00.000Z to 2021-08-16T04:00:00.000Z
5985 DOT/USDT candles in total from 2020-12-09T05:00:00.000Z to 2021-08-16T04:00:00.000Z
6984 DOT/USDT candles in total from 2020-10-28T13:00:00.000Z to 2021-08-16T04:00:00.000Z
7984 DOT/USDT candles in total from 2020-09-16T21:00:00.000Z to 2021-08-16T04:00:00.000Z
8678 DOT/USDT candles in total from 2020-08-18T23:00:00.000Z to 2021-08-16T04:00:00.000Z
Saved 8678 candles from 2020-08-18T23:00:00.000Z to 2021-08-16T04:00:00.000Z to ./dot_usdt_1h.csv


In [12]:
for filename in ['./ada_usdt_1h.csv', './xrp_usdt_1h.csv', './dot_usdt_1h.csv']:
    df = pd.read_csv('../data/0_raw/Binance/' + filename, header=None)
    df.columns = ['Timestamp', 'Open', 'High', 'Low', 'Close', 'Volume']
    df.set_index('Timestamp', inplace=True)
    df.index = df.index /1000
    df["Date"] = pd.to_datetime(df.index, unit='s')
    df.to_csv("../data/0_raw/" + filename)